## Распознование текста из микрофона

In [ ]:
sudo apt-get install portaudio19-dev

In [1]:
!environment/bin/pip install SpeechRecognition
!environment/bin/pip install PyAudio

In [2]:
import speech_recognition as sr

In [3]:
recognizer = sr.Recognizer()

In [4]:
recognizer.listen??

In [4]:
with sr.Microphone() as micro:
    print(1)
    audio = recognizer.listen(micro)
    print(2)

1
2


In [54]:
with open('data/microphone-results.wav', "wb") as f:
    f.write(audio.get_wav_data())

In [5]:
recognizer.recognize_google(audio, language='en-EN')

'speak speak speak speak speak speak'

## Распознование текста из аудиофайла

In [31]:
recognizer = sr.Recognizer()

In [58]:
with sr.AudioFile('data/microphone-results.wav') as source:
    audio1 = recognizer.record(source)

In [59]:
recognizer.recognize_google(audio1, language='en-EN')

'this is a sound detection chick 111 detection check'

## Text topic extraction

In [36]:
!py3_env/bin/pip install spacy
!py3_env/bin/pip install pymorphy2

In [37]:
import spacy

In [39]:
# spacy.load('ru')

In [60]:
from spacy.lang.en import English

In [61]:
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [63]:
text = recognizer.recognize_google(audio, language='en-EN')
tokenize(text)

['this', 'is', 'a', 'sound', 'detection', 'chick', '111', 'detection', 'check']

In [3]:
!py3_env/bin/pip install nltk

/bin/sh: 1: py3_env/bin/pip: not found


In [2]:
import nltk
nltk.download('wordnet')

ModuleNotFoundError: No module named 'nltk'

In [48]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [78]:
for token in tokenize(text):
    print(get_lemma(token), get_lemma2(token))

this this
be is
a a
sound sound
detection detection
chick chick
111 111
detection detection
check check


In [67]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [72]:
list(en_stop)[:5]

['herself', 'such', 'here', 'y', 'on']

In [90]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    return [get_lemma(token) for token in tokens 
            if len(token) > 4 and token not in en_stop]

In [91]:
text_data = [prepare_text_for_lda(text)]
text_data

[['sound', 'detection', 'chick', 'detection', 'check']]

In [82]:
!py3_env/bin/pip install gensim

In [86]:
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in text_data]
print(corpus)

Dictionary(4 unique tokens: ['check', 'chick', 'detection', 'sound'])
[[(0, 1), (1, 1), (2, 2), (3, 1)]]


In [89]:
dictionary.doc2bow?? # document to bag of words

In [88]:
import gensim

NUM_TOPICS = 2
NUM_WORDS = 3

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.251*"detection" + 0.250*"check" + 0.250*"sound"')
(1, '0.359*"detection" + 0.214*"chick" + 0.214*"sound"')


## загрузка аудиофайла из youtube

In [31]:
!py3_env/bin/pip install youtube_dl
!py3_env/bin/pip install oauth2client
!py3_env/bin/pip install pydub

In [29]:
import youtube_dl

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192'
    }],
    'outtmpl': '1',
    'prefer_ffmpeg': True,
    'keepvideo': False
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=J5O5iRpLXKY&t=637s'])

[youtube] J5O5iRpLXKY: Downloading webpage
[youtube] J5O5iRpLXKY: Downloading video info webpage
[download] 1 has already been downloaded
[download] 100% of 15.11MiB
[ffmpeg] Destination: wav
Deleting original file 1 (pass -k to keep)


In [56]:
from pydub import AudioSegment
t1 = 110 * 1000 #Works in milliseconds
t2 = 200 * 1000
newAudio = AudioSegment.from_wav("data/1.wav")
newAudio = newAudio[t1:t2]
newAudio.export('data/3.wav', format="wav")

<_io.BufferedRandom name='data/3.wav'>

In [57]:
recognizer = sr.Recognizer()
with sr.AudioFile('data/3.wav') as source:
    audio1 = recognizer.record(source)
recognizer.recognize_google(audio1, language='ru_RU')

'чтобы после очень крутой 18 года попробовать себя на западе А это произошло наверное летом летом 18 летом 18 года Так как только артист увидел офсет увидел эту ссылку Он резко отправил своим продюсером и сказал у меня переписку показывали потом эти же продюсер искал это моё секретное оружие Давайте звать парня и потихоньку очень медленно начинался запускается процесс То есть как бы это ни было моментально там о Попов и мы уже Я уже тут В начале обсуждал Ну поскольку у них стартовал тур совместно с дрейком не улавливает of цитата 11 чуваков из группы migos когда архитектуре это всегда любой клип запись альбома идёт очень медленно потому что они ежедневно просто путешествую фотосессии концерты в каждом городе по два по три концерта все солдаут были и наш тоже клип он сильно двигался медленно но ближе наверное в октябре уже нам Мне позвонили сказали всё Поехали Давай всё снимать придумывай идею У артиста из Кто такие заметную столько-то заметок что он хочет видеть и мы сейчас я с этим св